In [1]:
!pip3 install chromedriver_installer \
    --install-option="--chromedriver-version=2.10" \
    --install-option="--chromedriver-checksums=4fecc99b066cb1a346035bf022607104,058cd8b7b4b9688507701b5e648fd821"
import pandas as pd
import re
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

/home/linuxbrew/.linuxbrew/opt/python@3.8/lib/python3.8/site-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)


In [2]:
gold = pd.read_csv('C_20201107_xauusd_price.csv')
gold['date'] = pd.to_datetime(gold['date'])

### Getting USD Liquidity data from the FED

## M1 (USD B)

In [3]:
# Downloading M1 Data in json format
series_id = 'M1'
api_url =  f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key=1c3477ff38d635714ac6c42a9f9f8e3d&file_type=json'
m1 = requests.get(api_url)
m1_data = m1.json()

# Cleaning the data

df_m1 = pd.DataFrame(m1_data['observations'])
df_m1 = df_m1.drop(columns = ['realtime_start', 'realtime_end'])
df_m1['date'] = pd.to_datetime(df_m1['date'])
df_m1['value'] = df_m1['value'].astype(float)
df_m1.rename(columns={'value':'M1 USD (B)'}, inplace = True)
df_m1.index = df_m1.date

##  M2  (USD B)

In [4]:
# Downloading M2 Data in json format

series_id = 'M2'
api_url =  f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key=1c3477ff38d635714ac6c42a9f9f8e3d&file_type=json'
m2 = requests.get(api_url)
m2_data = m2.json()

# Cleaning the data

df_m2 = pd.DataFrame(m2_data['observations'])
df_m2 = df_m2 = df_m2.drop(columns = ['realtime_start', 'realtime_end'])
df_m2['date'] = pd.to_datetime(df_m2['date'])
df_m2['value'] = df_m2['value'].astype(float)
df_m2.rename(columns={'value':'M2 USD (B)'}, inplace = True)
df_m2.index = df_m2.date
type(df_m2['date'][1])

pandas._libs.tslibs.timestamps.Timestamp

## Merging the DataFrames

In [5]:
m1_m2 = df_m1.merge(df_m2, how='inner', left_on=df_m1['date'], right_on=df_m2['date'])
m1_m2

,key_0,date_x,M1 USD (B),date_y,M2 USD (B)
0,1980-11-03,1980-11-03,409.2,1980-11-03,1591.4
1,1980-11-10,1980-11-10,410.4,1980-11-10,1592.9
2,1980-11-17,1980-11-17,411.3,1980-11-17,1596.3
3,1980-11-24,1980-11-24,410.8,1980-11-24,1597.2
4,1980-12-01,1980-12-01,410.5,1980-12-01,1596.1
...,...,...,...,...,...
2082,2020-09-28,2020-09-28,5500.2,2020-09-28,18643.3
2083,2020-10-05,2020-10-05,5507.7,2020-10-05,18689.1
2084,2020-10-12,2020-10-12,5603.9,2020-10-12,18795.6
2085,2020-10-19,2020-10-19,5545.7,2020-10-19,18815.3


In [6]:
m1_m2 = m1_m2.drop(columns = ['key_0','date_y'])
gold_m = gold.merge( m1_m2, how='inner', left_on=gold['date'], right_on=m1_m2['date_x'])

In [7]:
gold_m = gold_m.drop(columns =['key_0','date_x'])

In [8]:
gold_m.to_csv('E_20201109_xauusd_price.csv')